# Homework 5

Alles Rebel
Computational Science PhD



## Intoduction!
This homework looks at constrained optimization. We'll explore implementing the algorithms descibed in K&W and see how they work on real hardware! We'll collect data and discuss any interesting bits. 

Didn't attempt the extra credit - just focused on getting the minimum done for a grad student.


## Pre-Reqs

Like other homeworks, we'll start by getting the kernel set up with the right dependencies. Running this once should do the trick for the rest of the homework;

In [1]:
import Pkg; 
Pkg.add("Distributions")
Pkg.add("PrettyTables")
Pkg.add("Plots")
Pkg.add("ForwardDiff")
Pkg.add("Optim")

using Printf
using LinearAlgebra
using ForwardDiff
using Random
using Distributions
using Statistics
using Dates
using PrettyTables
using Optim
using Plots

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


## Problem 1

The goal of this assignment is to apply a gradient solver to a constrainted optimization problem. In this case the problem needs to be atleast 10 dimensions. The method I'll stick with is the first one suggested by the assignment: the method of Augmented Lagrangian! And we'll apply it to Booth's Function, just updated to use 10 variables instead of 2 via the chaining method introduced in HW2, 1C.

### Broad Overview of Augmented Lagrangians
In general, these methods solve optimization problems with constraints by incorporating the constraints as additional terms in the objective function. These additional terms involve new parameters, known as Lagrange multipliers, which help reformulate the constrained problem (involving equalities or inequalities) into an unconstrained one.

To ensure numerical stablity - Augmented lagrangians intoduce a penality term to the discourage constraint violations. The optimization alternates between minimizing the augmented Lagrangian and updating the multipliers and penalty parameters iteratively. That's what we'll be implementing below!

### Booth's Function!

We'll reuse the code I wrote from the earlier homework - line for line. Originally designed to be used with existing autodiff (forwardiff in julia) and Optim package. It will suit our purposes for this homework. 

In [2]:
# Booth's function - generalized for any number of variables
# Uses the method suggested by Homework2 Problem 1C to chain variables
# pair wise to extend from 2D -> ND
# I wrote this for HW2
function booths_n(x; n = 10)
    result = 0.0
    # same rationale for ending at n-1, we need the nth element for n-1
    for i in 1:(n - 1)
        result += ((x[i] + 2*x[i+1] - 7)^2 + (2*x[i] + x[i+1] - 5)^2)
    end
    return result
end

# Same with this - since the methods of HW needed gradients
# we'll need this as well for this HW
function booths_n_gradient(x; n = length(x))
    grad = zeros(length(x)) # modified to be typeless
    
    # Special case for the first element
    grad[1] = 2*(x[1] + 2*x[2] - 7) + 4*(2*x[1] + x[2] - 5)
    
    # Compute the gradient for the 2:n-1 elements
    for i in 2:(n-1)
        grad[i] = 2*(x[i]+2*x[i+1]-7) + 4*(2*x[i] + x[i+1] - 5) + 4*(x[i-1] + 2*x[i] - 7) + 2*(2*x[i-1] + x[i] - 5)
    end

    # Special case for the last element (n-th element)
    grad[n] = 4*(x[n-1] + 2*x[n] - 7) + 2*(2*x[n-1] + x[n] - 5) 
    
    return grad
end

# since this is a constraints optimization problem, we'll add something here

# Define the constraint h(x)
function h(x)
    return [sum(x)]  # For example, we can constrain the sum of x to zero
end    

h (generic function with 1 method)

### Implemention of augmented lagrangian method

This will be a bit involved, but in general - it's the following steps:
- We'll attempt to use Julia's Optim library for the minimization method
- We'll instrument up the result to match the results from earlier
(function calls, grad calls, convergement measure, optimization point)

In [3]:
# Augmented Lagrangian Method using Optim.optimize
# Based upon the code in Algorithms for Optimization (Algorithm 10.2 of K&W)
# Modified to keep track of function and grad calls
function augmented_lagrange_method(f, h, x0, k_max; ρ=1.0, γ=2.0, tol=1e-6)
    x = x0
    λ = zeros(length(h(x)))               # Initialize Lagrange multipliers
    old_f_value = f(x)                    # Track old objective value
    convergence_measure = 0.0             # Convergence measure

    total_f_calls = 0   # Total function evaluations
    total_g_calls = 0   # Total gradient evaluations

    for k in 1:k_max
        # Define penalty function
        function p(x)
            λ_promoted = convert.(eltype(x[1]), λ)
            return (ρ / 2) * sum(h(x).^2) - dot(λ_promoted, h(x))
        end

        # Define augmented objective function
        function augmented_objective(x)
            return f(x) + p(x)
        end

        # Optimize using Optim.optimize
        result = optimize(augmented_objective, x, method=LBFGS(); autodiff=:forward)
        x = Optim.minimizer(result)

        # Accumulate function and gradient evaluations
        total_f_calls += result.f_calls
        total_g_calls += result.g_calls

        # Update convergence metrics
        new_f_value = f(x)
        convergence_measure = abs(new_f_value - old_f_value)
        old_f_value = new_f_value

        # Check for convergence
        if convergence_measure < tol
            break
        end

        # Update Lagrange multipliers and penalty parameter
        λ .= λ .- ρ * ForwardDiff.value.(h(x))
        ρ *= γ
    end

    return x, total_f_calls, total_g_calls
end

augmented_lagrange_method (generic function with 1 method)

### Experimental Methodology

We'll utilize the same testing methodology as the HW2, generating a BUNCH of random vectors to start from. And then running the method across each of the vectors, recording how many function calls, gradient calls, exectuation time and error from what the ideal solution would have been. We'll compare the above method against the earlier line search method.

First we'll get the environment ready to do the experiment - how we'll generate everything an calculate the answers. 

In [4]:
# The code we used earlier in HW2! This will be what we'll compare against
# I wrote this code with the book as the primary sources
# Conjugate Gradient Descent method (with function and gradient counts)
# Updated tracking - and fixed accounting issues missed in originial HW2 implementation
mutable struct ConjugateGradientDescent
    d::Vector{Float64}
    g::Vector{Float64}
    iterations::Int
    func_calls::Int
    grad_calls::Int
    converged::Bool
end

function init!(M::ConjugateGradientDescent, f, ∇f, x)
    M.g = ∇f(x)
    M.d = -M.g
    M.iterations = 0
    M.func_calls = 1
    M.grad_calls = 1
    M.converged = false
    return M
end

function backtracking_line_search(f, ∇f, x, d, α; p=0.5, β=1e-4)
    y, g = f(x), ∇f(x)
    func_calls = 1
    grad_calls = 1
    while f(x + α*d) > y + β*α*(g⋅d)
        α *= p
        func_calls += 1
    end
    return α, func_calls, grad_calls
end

function step!(M::ConjugateGradientDescent, f, ∇f, x)
    d, g = M.d, M.g
    α, f_calls_ls, g_calls_ls = backtracking_line_search(f, ∇f, x, d, 1.0)  # initial α can be 1.0
    x_new = x + α * d
    M.func_calls += f_calls_ls
    M.grad_calls += g_calls_ls
    M.func_calls += 1  # For evaluating f at x_new
    g_new = ∇f(x_new)
    M.grad_calls += 1
    β = max(0, dot(g_new, g_new - g) / dot(g, g))
    d_new = -g_new + β * d
    M.d = d_new
    M.g = g_new
    M.iterations += 1
    if norm(g_new) < 1e-6
        M.converged = true
    end
    return x_new
end

step! (generic function with 1 method)

In [5]:
## The code I wrote to generate the starting points from HW2
function generate_initial_points(left_limit=-10.0, right_limit=10.0, samples=100, dims=10)
    # Generate 100 random real-valued starting points as specified
    initial_points = [] 

    for i in 1:samples # defaults to 100 samples
        this_sample = []
        
        for j in 1:dims
            # y is uniformly picked in [-10.0, 10.0] by default
            # note, rand is already uniform, just need to cap the limits
            y = rand(Uniform(left_limit, right_limit))
            # choose + or - with equal probability, we'll use bitrand (50/50)
            sign = bitrand() == 0 ? -1 : 1
            x = sign * exp(y)
            push!(this_sample, x)
        end
        
        push!(initial_points, this_sample)
    end

    return initial_points
end



generate_initial_points (generic function with 5 methods)

Running the experiment is similiar to how we did it before - we generate 100 random sample starting points. And for each, we run both methods to see how well the methods perform in terms of time and function/gradient calls.

In [6]:
## Experiment code is effectively the same as previous homeworks with renamed variables
# Handles the time keeping of each optimizization method

# Generate initial points for Booth's function
initial_points_booths = generate_initial_points()  # Using multiple samples, 100 as we did before

# Get the numberical solution using LBFGS (for comparison), using built in solver in Optim
booths_opt_res = optimize(booths_n, convert(Vector{Float64}, initial_points_booths[1]), LBFGS(); autodiff=:forward)
booths_opt_min_x = Optim.minimizer(booths_opt_res)

# Lists to store results
results_conj_booths = []
results_aug_lag_booths = []

# Run the optimizer for each method
for (method_name, results) in zip(["Conjugate Gradient Descent", "Augmented Lagrangian Method"], [results_conj_booths, results_aug_lag_booths])
    for x0 in initial_points_booths
        x0 = convert(Vector{Float64}, x0)
        start_time = time()
        if method_name == "Conjugate Gradient Descent"
            # Initialize CGD
            M = ConjugateGradientDescent(zeros(length(x0)), zeros(length(x0)), 0, 0, 0, false)
            init!(M, booths_n, booths_n_gradient, x0)
            iteration_limit = 1000
            x = x0
            for i in 1:iteration_limit
                x = step!(M, booths_n, booths_n_gradient, x)
                if M.converged
                    break
                end
                if any(isnan, x) || any(isinf, x)
                    break
                end
            end
            elapsed_time = time() - start_time
            f_calls, g_calls, iterations, converged = M.func_calls, M.grad_calls, M.iterations, M.converged
            absolute_error = norm(x - booths_opt_min_x, Inf)
            push!(results, (absolute_error, converged, f_calls, g_calls, elapsed_time))
        elseif method_name == "Augmented Lagrangian Method"
            # Use augmented Lagrangian method
            k_max = 10  # Maximum number of outer iterations
            x_opt, total_f_calls, total_g_calls = augmented_lagrange_method(booths_n, h, x0, k_max)
            elapsed_time = time() - start_time
            absolute_error = norm(x_opt - booths_opt_min_x, Inf)
            push!(results, (absolute_error, total_f_calls, total_g_calls, elapsed_time))
        end
    end
end


In [7]:
# Function to compute statistics and display results in a table
# Adapted from previous homeworks (using PrettyTable to show the info!)
function display_results_table(method_names, results_list)
    # Collect data rows
    data_rows = []

    for (method_name, results) in zip(method_names, results_list)
        errors = [res[1] for res in results]
        times = [res[2] for res in results]
        evals = [res[3] for res in results]

        mean_error = mean(errors)
        std_error = std(errors)
        mean_time = mean(times)
        std_time = std(times)
        mean_evals = mean(evals)
        std_evals = std(evals)

        # Create data row using hcat
        data_row = hcat(
            method_name,
            @sprintf("%.4e ± %.4e", mean_error, std_error),
            @sprintf("%.2f ± %.2f", mean_evals, std_evals),
            @sprintf("%.2f ± %.2f", mean_time, std_time)
        )

        push!(data_rows, data_row)
    end

    # Combine data rows using vcat
    table = vcat(data_rows...)

    # Display the table using PrettyTables
    headers = ["Method", "Absolute Error", "Evaluations", "Time (ms)"]
    pretty_table(table, header = headers, title = "Optimization Results (mean ± std)")
end

# Prepare the data for displaying
method_names = ["Conjugate Gradient Descent", "Augmented Lagrangian Method"]
results_list = [results_conj_booths, results_aug_lag_booths]

# Call the function to display the results table
display_results_table(method_names, results_list)

Optimization Results (mean ± std)
┌─────────────────────────────┬─────────────────────────┬──────────────────┬───────────────┐
│                      Method │          Absolute Error │      Evaluations │     Time (ms) │
├─────────────────────────────┼─────────────────────────┼──────────────────┼───────────────┤
│  Conjugate Gradient Descent │ 4.2703e-08 ± 3.0150e-08 │ 1388.95 ± 325.87 │   1.00 ± 0.00 │
│ Augmented Lagrangian Method │ 5.7023e+00 ± 4.5574e-10 │    177.29 ± 3.11 │ 177.29 ± 3.11 │
└─────────────────────────────┴─────────────────────────┴──────────────────┴───────────────┘


### Results and Discussion

I suspect there's a bug in the augmented lagragian implementation - when combing through a few runs by printing the results - it seems that Augmented Lagrangian tends to finish faster than Conjugate Gradient; there's a bunch of runs that end up taking a long time due to poor line searching. However, overall function and gradient evaltions are significantly lower in Augmented Lagrangian vs Conjugate Gradient Descent. Another interesting bit of information is the error - the implementation error results in the absolute error being quite large. If more time was available, I would have dug into this more.

Another interesting point is that the Conjugate Gradient Descent didn't take into account the constraint! So it's possible Agumented Lagrangian is not only faster - but more accurate given the made up constraint I put in. Since they are optimizing different functions - the evaluations make a bit more sense to use rather than time or error. I could have also selected a better constraint.

Note: the absolute error isn't also useful either - it doesn't take into account the constraint! So really, it's just evaluations and timing.

## Problem 2

This problem we're going to use a stocastic solver, simulated annealing, on the Booth function! Atleast this time we can compare the two methods head to head, both will operate on the same constraints. Very simliar experimental set up as Problem 1. 

We'll utilize http://julianlsolvers.github.io/Optim.jl/stable/algo/simulated_annealing/ for most of the information. Our focus will be to get it instrumented as much as we can, and collect some stats!

The goal will be to apply it against a 10 variable Booth's equation. And for fun, we'll compare it against the Augmented Lagrangian method from Problem 1.

### Experiment

It'll be the same drill as before - collecting 100 random starting points and running through those for each method. We'll reuse as much code as possible from previous problem and homeworks. We'll compare against the solution created by using the built in LBFGS optimizer in Optim. Same as before regarding the conversion of 2 Variable booths to 10 variable booths (via variable chaining by pairs).

In [8]:
## Experiment code is effectively the same as previous homeworks with renamed variables
# Handles the time keeping of each optimizization method

# Generate initial points for Booth's function
initial_points_booths = generate_initial_points()  # Using multiple samples, 100 as we did before

# Get the numberical solution using LBFGS (for comparison), using built in solver in Optim
booths_opt_res = optimize(booths_n, convert(Vector{Float64}, initial_points_booths[1]), LBFGS(); autodiff=:forward)
booths_opt_min_x = Optim.minimizer(booths_opt_res)

# Lists to store results
results_sim_ann_booths = []
results_aug_lag_booths = []

# Run the optimizer for each method
for (method_name, results) in zip(["Simulated Annealing", "Augmented Lagrangian Method"], [results_sim_ann_booths, results_aug_lag_booths])
    for x0 in initial_points_booths
        x0 = convert(Vector{Float64}, x0)
        start_time = time()
        if method_name == "Simulated Annealing"
            # Use Optim.optimize with SimulatedAnnealing
            result = optimize(booths_n, x0, method=SimulatedAnnealing())
            x = Optim.minimizer(result)
            elapsed_time = time() - start_time
            f_calls = result.f_calls
            total_evals = f_calls
            absolute_error = norm(x - booths_opt_min_x, Inf)
            push!(results, (absolute_error, total_evals, elapsed_time*1e3))
        elseif method_name == "Augmented Lagrangian Method"
            # Use augmented Lagrangian method
            k_max = 10  # Maximum number of outer iterations
            x_opt, total_f_calls, total_g_calls = augmented_lagrange_method(booths_n, h, x0, k_max)
            elapsed_time = time() - start_time
            total_evals = total_f_calls + total_g_calls
            absolute_error = norm(x_opt - booths_opt_min_x, Inf)
            push!(results, (absolute_error, total_evals, elapsed_time*1e3))
        end
    end
end

# Function to compute statistics and display results in a table
function display_results_table(method_names, results_list)
    # Collect data rows
    data_rows = []
    for (method_name, results) in zip(method_names, results_list)
        errors = [res[1] for res in results]
        evals = [res[2] for res in results]
        times = [res[3] for res in results]
        mean_error = mean(errors)
        std_error = std(errors)
        mean_evals = mean(evals)
        std_evals = std(evals)
        mean_time = mean(times)
        std_time = std(times)
        # Create data row
        data_row = hcat(
            method_name,
            @sprintf("%.4e ± %.4e", mean_error, std_error),
            @sprintf("%.2f ± %.2f", mean_evals, std_evals),
            @sprintf("%.4f ± %.4f", mean_time, std_time)
        )
        push!(data_rows, data_row)
    end
    # Combine data rows using vcat
    table = vcat(data_rows...)
    # Display the table using PrettyTables
    headers = ["Method", "Absolute Error", "Evaluations", "Time (ms)"]
    pretty_table(table, header = headers, title = "Optimization Results (mean ± std)")
end

# Prepare the data for displaying
method_names = ["Simulated Annealing", "Augmented Lagrangian Method"]
results_list = [results_sim_ann_booths, results_aug_lag_booths]

# Call the function to display the results table
display_results_table(method_names, results_list)


Optimization Results (mean ± std)
┌─────────────────────────────┬─────────────────────────┬────────────────┬──────────────────┐
│                      Method │          Absolute Error │    Evaluations │        Time (ms) │
├─────────────────────────────┼─────────────────────────┼────────────────┼──────────────────┤
│         Simulated Annealing │ 7.5706e+03 ± 6.1135e+03 │ 1001.00 ± 0.00 │ 3.3462 ± 31.2140 │
│ Augmented Lagrangian Method │ 5.7023e+00 ± 4.5883e-10 │  354.42 ± 6.15 │  0.3258 ± 0.0164 │
└─────────────────────────────┴─────────────────────────┴────────────────┴──────────────────┘


### Results and Discussion

To be honest, based on the number of evaluations - I would have expected Simulated Annealing to take MUCH longer. It's possible it's using the same evaluations over and over, meaning an evaluation's result may be a 'hot' cache item - causing really quick fetching. This makes sense in the context of simulated annealing - it's supposed to perfer the more likely outcomes - and this would be the definition of a cache friendly operation. However, this would require further investigation to verify.

Augmented Lagrangian performs the same as the previous method (which it should! no implementation changes). It's atleast consistent with similiar timings and error. The error is actually much larger with Simulated Annealing. But I don't trust the error much given that the solution found by the optimizer doesn't take into account my made up constraint. 